In [1]:
import pandas as pd
import numpy as np

In [59]:
data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")

C:\Users\default.DESKTOP-GAN0M7C\AppData\Local\Temp\ipykernel_4332\179680575.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

### Data Cleaning & Transformations

In [61]:
def preprocess_orders(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the orders dataset:
    - Drop irrelevant or high-cardinality columns
    - Rename columns if needed
    - Extract useful features from 'Date'
    - Create derived features like day, month, weekday, weekend flag
    """
    
    # 1. Drop unwanted columns if they exist
    DROP_COLS = [
        "index",
        "Order ID",           # pure identifier
        "SKU",                # high cardinality ID
        "promotion-ids",      # sparse / leakage-prone
        "Unnamed: 22",
        "ASIN",
        "fulfilled-by",
        "ship-postal-code",
        "currency",
        "Size"
    ]
    
    data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])
    
    # 2. Rename columns if necessary
    if 'Sales Channel ' in data.columns:
        data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)
    
    # 3. Convert 'Date' to datetime safely
    if 'Date' in data.columns:
        data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")
        
        # Extract date-related features
        data["order_day"] = data["Date"].dt.day_name()
        data["order_month"] = data["Date"].dt.month_name()
        data["order_weekday"] = data["Date"].dt.weekday
        data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)
        
        # Drop original Date column
        data = data.drop(columns=["Date"])
    
    return data


In [62]:
data = preprocess_orders(data)
print(data.head())

                         Status Fulfilment Sales Channel ship-service-level  \
0                     Cancelled   Merchant     Amazon.in           Standard   
1  Shipped - Delivered to Buyer   Merchant     Amazon.in           Standard   
2                       Shipped     Amazon     Amazon.in          Expedited   
3                     Cancelled   Merchant     Amazon.in           Standard   
4                       Shipped     Amazon     Amazon.in          Expedited   

     Style       Category Courier Status  Qty  Amount    ship-city  \
0   SET389            Set            NaN    0  647.62       MUMBAI   
1  JNE3781          kurta        Shipped    1  406.00    BENGALURU   
2  JNE3371          kurta        Shipped    1  329.00  NAVI MUMBAI   
3    J0341  Western Dress            NaN    0  753.33   PUDUCHERRY   
4  JNE3671            Top        Shipped    1  574.00      CHENNAI   

    ship-state ship-country    B2B order_day order_month  order_weekday  \
0  MAHARASHTRA           IN  

C:\Users\default.DESKTOP-GAN0M7C\AppData\Local\Temp\ipykernel_4332\1854827303.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")


In [63]:
data.Amount.max()

5584.0

In [64]:
data.Amount.min()

0.0

In [65]:
# create 3 classes using quantiles (balanced classes)
data["sales_class"] = pd.qcut(
    data["Amount"],
    q=3,
    labels=["Low", "Medium", "High"]
)

# check distribution
data["sales_class"].value_counts()


sales_class
Medium    41555
Low       40498
High      39127
Name: count, dtype: int64

Quantiles gives -> balanced classes, which avoids the avoids class imbalance

Standard practice for turning regression -> classification

### Feature Engineering

In [66]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month', 'sales_class'],
      dtype='object')


In [67]:
# drop Amount, as it is directly influencing the target
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

In [68]:
# dropping the target variable
categorical_cols = categorical_cols.drop(['sales_class'])
categorical_cols

Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from category_encoders.count import CountEncoder

def build_classification_preprocessor(num_cols, cat_cols):
    """
    Preprocessor for tree-based classification models:
    - Numerical: median imputation
    - Categorical: frequency (count) encoding
    """
    
    # 1. Numerical pipeline
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median"))
    ])
    
    # 2. Categorical pipeline
    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", CountEncoder(normalize=True))  # LightGBM/XGB-friendly
    ])
    
    # 3. Combine into ColumnTransformer
    preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])
    
    return preprocessor


In [70]:
# for linear models -> scaling is on!
preprocessor = build_classification_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
)

In [71]:
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [72]:
data.isna().sum()

Status                   0
Fulfilment               0
Sales Channel            0
ship-service-level       0
Style                    0
Category                 0
Courier Status        6872
Qty                      0
Amount                7795
ship-city               33
ship-state              33
ship-country            33
B2B                      0
order_day                0
order_month              0
order_weekday            0
is_weekend               0
sales_class           7795
dtype: int64

In [73]:
data = data.dropna(subset=['sales_class'])
data.isna().sum()

Status                   0
Fulfilment               0
Sales Channel            0
ship-service-level       0
Style                    0
Category                 0
Courier Status        5136
Qty                      0
Amount                   0
ship-city               31
ship-state              31
ship-country            31
B2B                      0
order_day                0
order_month              0
order_weekday            0
is_weekend               0
sales_class              0
dtype: int64

In [74]:
# seperating the input and target variable from data
TARGET = "sales_class"   # regression target

X = data.drop(columns=[TARGET, 'Amount'])
y = data[TARGET]


In [75]:
y.isna().sum()

np.int64(0)

In [76]:
X.shape

(121180, 16)

In [77]:
y.shape

(121180,)

In [78]:
# seperating the input and target data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [79]:
X_train.shape

(96944, 16)

In [80]:
y_train.shape

(96944,)

In [84]:
# use actual class labels for stratification
X_train_sampled, _, y_train_sampled, _ = train_test_split(
    X_train,
    y_train,              # categorical target
    test_size=1-0.2,
    stratify=y_train,     # correct for classification
    random_state=42
)


In [86]:
X_train_sampled.head(2)

,Status,Fulfilment,Sales Channel,ship-service-level,Style,Category,Courier Status,Qty,ship-city,ship-state,ship-country,B2B,order_day,order_month,order_weekday,is_weekend
96159,Shipped - Picked Up,Merchant,Amazon.in,Standard,SET333,Set,Shipped,1,KOLHAPUR,MAHARASHTRA,IN,False,Saturday,June,5,1
127693,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3726,kurta,Shipped,1,GURUGRAM,HARYANA,IN,False,Thursday,January,3,0


In [87]:
# Fit on full training data and transform both train and test
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# For sampled training data, just transform (already fitted)
X_train_sampled_processed = preprocessor.transform(X_train_sampled)


### Model Training 

In [89]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Build LightGBM classifier (production-safe defaults)
lgbm_clf = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)


In [93]:

# Train on sampled processed data
lgbm_clf.fit(X_train_processed, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006018 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 524
[LightGBM] [Info] Number of data points in the train set: 96944, number of used features: 14
[LightGBM] [Info] Start training from score -1.128090
[LightGBM] [Info] Start training from score -1.097643
[LightGBM] [Info] Start training from score -1.070921


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [94]:

# Predict on full test set
y_pred = lgbm_clf.predict(X_test_processed)


C:\Users\default.DESKTOP-GAN0M7C\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [95]:

#  Evaluate
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[6895  211  645]
 [ 233 7445  474]
 [ 842 1003 6488]]

Classification Report:
               precision    recall  f1-score   support

        High       0.87      0.89      0.88      7751
         Low       0.86      0.91      0.89      8152
      Medium       0.85      0.78      0.81      8333

    accuracy                           0.86     24236
   macro avg       0.86      0.86      0.86     24236
weighted avg       0.86      0.86      0.86     24236



overall accuracy is 86%, which is considered as good!